Assignment 2: Caitlin Box

In [91]:
import pygame
import moderngl
import numpy

In [92]:
WIDTH = 600
HEIGHT = 600
pygame.init()

(5, 0)

In [93]:
pygame.display.set_mode((HEIGHT, WIDTH), flags= pygame.OPENGL | pygame.DOUBLEBUF)
pygame.display.set_caption(title="Assignment 2")
gl = moderngl.get_context()

In [94]:
# two different triangles to make diamond 
position_data = [
    #x    y 
    0.0, 0.8,
    0.8, 0.0,
    -0.8, 0.0,

    0.0, -0.8,
    0.8, 0.0,
    -0.8, 0.0
]
triangle_vertex_positins = numpy.array(position_data).astype("float32")
triangle_position_buffer = gl.buffer(triangle_vertex_positins)

In [95]:
fragment_shader_code = '''
#version 330 core

out vec4 color; 
uniform vec3 inColor; 

void main(){
    color = vec4(inColor, 0);
}
'''

vertex_shader_code = '''
#version 330 core 

layout (location = 0) in vec2 position; 
uniform float scale; 
uniform float distance;
uniform float angle; 

void main() {
    float angleR = radians(-angle); 
    vec2 d_vector =  distance*vec2(cos(angleR), sin(angleR));
    vec2 P = position*scale + d_vector;
    gl_Position = vec4(P, 0.0, 1.0); 
}
'''
vertex_shader_code_line = '''
#version 330 core 
 
uniform float distance;
uniform float angle;  

void main() {
    vec2 position = vec2(0); 
    if (gl_VertexID > 0){
        float angleR = radians(-angle); 
        position =  distance*vec2(cos(angleR), sin(angleR));
    }
    gl_Position = vec4(position, 0.0, 1.0); 
}
'''

In [96]:
program = gl.program(
    vertex_shader = vertex_shader_code,
    fragment_shader = fragment_shader_code
)
program_line = gl.program(
    vertex_shader = vertex_shader_code_line,
    fragment_shader = fragment_shader_code
)

renderable = gl.vertex_array(program,
    [( triangle_position_buffer, "2f", "position")])

line_renderable = gl.vertex_array(program_line, [])

In [97]:
running = True
clock = pygame.time.Clock()
angle = 0
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key==27):
            running = False
    
    gl.clear(0.5, 0.5, 0) # background color 
    
    program ['scale'] = 0.1
    program ['distance'] = 0
    program['inColor'] = [1,1,0]
    program ['angle'] = 0
     # default param -  didnt have to put these in, default but if u put in 3 itll ignore the other 3 
    renderable.render(moderngl.TRIANGLES, vertices = 6) 

    # this will change location of the second diamond 
    program ['distance'] = 0.8
    program ['angle'] = angle
    renderable.render(moderngl.TRIANGLES, vertices = 6) 

    # without setting uniform variables, nothing shows up 
    program_line ['distance'] = 0.8
    program_line ['angle'] = angle
    line_renderable.render(moderngl.LINES, vertices = 2)

    pygame.display.flip()
    clock.tick(60)
    angle = angle + 0.1
    if angle > 360:
        angle = 0

pygame.quit()

scale the geometry 
uniform global to vertex (uniform float scale;)
position * scale 
    geometry will disappear, did not set uniform before the render 
program ['scale'] = 0.1 -> scale is the name and then the 0.1 is sizing it differently 
___
we need two of them but if we render it again itll be on top of the other one 
second one is displaced, we will use displacement vector 
___
